In [1]:
import pandas as pd

data = pd.read_csv("train.csv")
data

,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,...,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Response
0,1,A8,26,0.230769,2,3,1,0.492537,0.563636,0.225941,...,0,0,0,0,0,0,0,0,0,7
1,1,D2,26,1.000000,3,3,1,0.671642,0.781818,0.326360,...,0,0,0,0,0,0,0,0,0,6
2,1,D2,26,0.230769,2,3,1,0.626866,0.745455,0.393305,...,0,0,0,0,0,0,0,0,0,1
3,1,D4,26,0.230769,2,3,1,0.268657,0.690909,0.246862,...,0,0,0,0,0,0,0,0,0,4
4,1,E1,10,0.230769,2,3,1,0.641791,0.581818,0.466527,...,0,0,1,0,0,0,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58876,1,A1,26,0.051282,2,3,1,0.507463,0.781818,0.518828,...,0,0,1,0,0,0,0,0,1,2
58877,1,A1,26,0.128205,2,3,1,0.223881,0.709091,0.223849,...,0,0,0,0,0,0,0,0,0,8
58878,1,D3,26,0.076923,2,3,1,0.611940,0.745455,0.309623,...,0,0,0,1,0,0,0,0,0,8
58879,1,D3,26,1.000000,2,3,1,0.298507,0.690909,0.234310,...,0,0,0,0,0,0,0,0,0,8


In [2]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
from IPython.display import clear_output

npdata = data.to_numpy().copy()
npdata = np.delete(npdata, -1, 1)

labEncoder = LabelEncoder(); 

all_label = npdata.reshape(1, -1)[0]
labEncoder.fit(all_label.astype(str))

LabelEncoder()

In [3]:
for i in range(len(npdata)):
    if(i%500 == 0):
        clear_output()
        print(str(i)+'/'+str(len(npdata)))
    npdata[i] = labEncoder.transform(npdata[i].astype(str))

clear_output()
print(str(len(npdata))+'/'+str(len(npdata)))
npdata

58881/58881


array([[10152, 11311, 10539, ..., 0, 0, 0],
       [10152, 11326, 10539, ..., 0, 0, 0],
       [10152, 11326, 10539, ..., 0, 0, 0],
       ...,
       [10152, 11327, 10539, ..., 0, 0, 0],
       [10152, 11327, 10539, ..., 0, 0, 0],
       [10152, 11328, 10539, ..., 0, 0, 0]], dtype=object)

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(npdata, data['Response'].to_numpy())
y_test.shape

(14721,)

In [5]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler_standard = StandardScaler()
scaler_standard.fit(npdata)

X_scaled_train = scaler_standard.transform(X_train)
X_scaled_test = scaler_standard.transform(X_test)

In [6]:
from sklearn.neighbors import KNeighborsClassifier

knn100 = KNeighborsClassifier(n_neighbors=60)

In [7]:
pca = PCA(n_components=6)
pca.fit(X_scaled_train)
X_pca_train = pca.transform(X_scaled_train)
X_pca_test = pca.transform(X_scaled_test)
knn100.fit(X_pca_train, y_train)
knn100.score(X_pca_test, y_test)

0.4169553698797636

# Genre

In [20]:
table_predict = knn100.predict(X_pca_test)

In [21]:
table = pd.DataFrame()
series = pd.Series([0] * 8, index = range(1,9))
table['Male'] = series
table['Female'] = series
table['Sum'] = series

table.loc['Sum'] = [0, 0, 0]
table 

,Male,Female,Sum
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
5,0,0,0
6,0,0,0
7,0,0,0
8,0,0,0
Sum,0,0,0


In [23]:
for i in range(X_test.shape[0]):
    if(X_test[i][23] == 11335):
        table['Male'][table_predict[i]] += 1
        table['Male']['Sum'] += 1
    elif(X_test[i][23] == 11330):
        table['Female'][table_predict[i]] += 1
        table['Female']['Sum'] += 1
    table['Sum'][table_predict[i]] += 1
    table['Sum']['Sum'] += 1
table

,Male,Female,Sum
1,986,986,1972
2,898,898,1796
3,1,1,2
4,19,19,38
5,218,218,436
6,3554,3554,7108
7,857,857,1714
8,8188,8188,16376
Sum,14721,14721,29442


In [13]:
for i in range(data.shape[0]):
    if(data['InsuredInfo_7'][i] == 'Male'):
        table['Male'][data['Response'][i]] += 1
        table['Male']['Sum'] += 1
    elif(data['InsuredInfo_7'][i] == 'Female'):
        table['Female'][data['Response'][i]] += 1
        table['Female']['Sum'] += 1
    table['Sum'][data['Response'][i]] += 1
    table['Sum']['Sum'] += 1
table

,Male,Female,Sum
1,3660,2498,6158
2,3903,2592,6495
3,583,426,1009
4,821,599,1420
5,3136,2254,5390
6,8000,3136,11136
7,4996,2950,7946
8,8774,10553,19327
Sum,33873,25008,58881


In [91]:
T = 0

for g in ['Male', 'Female']:
    for i in range(1,9):
        Eij = (table['Sum'][i] * table[g]['Sum'])/table['Sum']['Sum']
        T += (table[g][i] - Eij)**2/Eij
        
T

2143.417337704642

# Religion

In [24]:
table_predict = knn100.predict(X_pca_test)

In [52]:
table = pd.DataFrame()
series = pd.Series([0] * 8, index = range(1,9))
table['Christian'] = series
table['Muslim'] = series
table['Jewish'] = series
table['Buddhist'] = series
table['Hindus'] = series
table['Other'] = series
table['Sum'] = series

table.loc['Sum'] = [0]*table.shape[1]
table 

,Christian,Muslim,Jewish,Buddhist,Hindus,Other,Sum
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0
Sum,0,0,0,0,0,0,0


In [29]:
for i in range(data.shape[0]):
    if(data['InsuredInfo_9'][i] == 'Christian'):
        table['Christian'][data['Response'][i]] += 1
        table['Christian']['Sum'] += 1
    elif(data['InsuredInfo_9'][i] == 'Muslim'):
        table['Muslim'][data['Response'][i]] += 1
        table['Muslim']['Sum'] += 1
    elif(data['InsuredInfo_9'][i] == 'Jewish'):
        table['Jewish'][data['Response'][i]] += 1
        table['Jewish']['Sum'] += 1
    elif(data['InsuredInfo_9'][i] == 'Buddhist'):
        table['Buddhist'][data['Response'][i]] += 1
        table['Buddhist']['Sum'] += 1
    elif(data['InsuredInfo_9'][i] == 'Hindus'):
        table['Hindus'][data['Response'][i]] += 1
        table['Hindus']['Sum'] += 1
    else:
        table['Other'][data['Response'][i]] += 1
        table['Other']['Sum'] += 1
    table['Sum'][data['Response'][i]] += 1
    table['Sum']['Sum'] += 1
table

,Christian,Muslim,Jewish,Buddhist,Hindus,Other,Sum
1,385,384,536,113,166,10732,12316
2,414,358,535,117,180,11386,12990
3,62,60,88,18,33,1757,2018
4,39,136,166,9,14,2476,2840
5,136,563,588,44,49,9400,10780
6,264,1061,1221,94,115,19517,22272
7,193,800,932,49,73,13845,15892
8,431,1905,2115,110,199,33894,38654
Sum,1924,5267,6181,554,829,103007,117762


In [32]:
T = 0

for g in table.keys():
    for i in range(1,9):
        Eij = (table['Sum'][i] * table[g]['Sum'])/table['Sum']['Sum']
        T += (table[g][i] - Eij)**2/Eij
        
T

1162.7044350127064

In [40]:
lab = []

for i in range(X_test.shape[0]):
    if X_test[i][25] not in lab:
        lab.append(X_test[i][25])

lab

[11337, 11331, 11336, 11333, 11324, 11318]

In [45]:
count = [0, 0, 0, 0, 0, 0]

for i in range(X_test.shape[0]):
    count[lab.index(X_test[i][25])] += 1

count

[11067, 185, 1270, 1552, 507, 140]

In [53]:
for i in range(X_test.shape[0]):
    if(X_test[i][25] == 11324):
        table['Christian'][table_predict[i]] += 1
        table['Christian']['Sum'] += 1
    elif(X_test[i][25] == 11336):
        table['Muslim'][table_predict[i]] += 1
        table['Muslim']['Sum'] += 1
    elif(X_test[i][25] == 11333):
        table['Jewish'][table_predict[i]] += 1
        table['Jewish']['Sum'] += 1
    elif(X_test[i][25] == 11331):
        table['Buddhist'][table_predict[i]] += 1
        table['Buddhist']['Sum'] += 1
    elif(X_test[i][25] == 11318):
        table['Hindus'][table_predict[i]] += 1
        table['Hindus']['Sum'] += 1
    else:
        table['Other'][table_predict[i]] += 1
        table['Other']['Sum'] += 1
    table['Sum'][table_predict[i]] += 1
    table['Sum']['Sum'] += 1
table

,Christian,Muslim,Jewish,Buddhist,Hindus,Other,Sum
1,59,59,101,25,17,725,986
2,40,66,102,18,12,660,898
3,0,0,0,0,0,1,1
4,0,4,0,1,0,14,19
5,11,18,25,1,3,160,218
6,135,309,375,55,43,2637,3554
7,29,63,88,10,7,660,857
8,233,751,861,75,58,6210,8188
Sum,507,1270,1552,185,140,11067,14721


In [54]:
T = 0

for g in table.keys():
    for i in range(1,9):
        Eij = (table['Sum'][i] * table[g]['Sum'])/table['Sum']['Sum']
        T += (table[g][i] - Eij)**2/Eij
        
T

102.43666051667181

# Ethnic group

In [8]:
table_predict = knn100.predict(X_pca_test)

In [14]:
lab = []

for i in range(data.shape[0]):
    if data['InsuredInfo_8'][i] not in lab:
        lab.append(data['InsuredInfo_8'][i])

lab

[nan,
 'Caucasian',
 'Black',
 'Arabic',
 'African-American',
 'Asian',
 'Hispanic',
 'Latino']

In [20]:
table = pd.DataFrame()
series = pd.Series([0] * 8, index = range(1,9))
table['Caucasian'] = series
table['Black'] = series
table['Arabic'] = series
table['African-American'] = series
table['Asian'] = series
table['Hispanic'] = series
table['Latino'] = series
table['Other'] = series
table['Sum'] = series

table.loc['Sum'] = [0]*table.shape[1]
table 

,Caucasian,Black,Arabic,African-American,Asian,Hispanic,Latino,Other,Sum
1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0
Sum,0,0,0,0,0,0,0,0,0


In [16]:
lab = []

for i in range(X_test.shape[0]):
    if X_test[i][24] not in lab:
        lab.append(X_test[i][24])

lab

[11337, 11323, 11332, 11317, 11314, 11312, 11334, 11313]

In [13]:
count = [0, 0, 0, 0, 0, 0, 0, 0]

for i in range(X_test.shape[0]):
    count[lab.index(X_test[i][24])] += 1

count

[12866, 242, 65, 988, 108, 215, 233, 4]

In [19]:
for i in range(data.shape[0]):
    if(data['InsuredInfo_8'][i] == 'Caucasian'):
        table['Caucasian'][data['Response'][i]] += 1
        table['Caucasian']['Sum'] += 1
    elif(data['InsuredInfo_8'][i] == 'Black'):
        table['Black'][data['Response'][i]] += 1
        table['Black']['Sum'] += 1
    elif(data['InsuredInfo_8'][i] == 'Arabic'):
        table['Arabic'][data['Response'][i]] += 1
        table['Arabic']['Sum'] += 1
    elif(data['InsuredInfo_8'][i] == 'African-American'):
        table['African-American'][data['Response'][i]] += 1
        table['African-American']['Sum'] += 1
    elif(data['InsuredInfo_8'][i] == 'Asian'):
        table['Asian'][data['Response'][i]] += 1
        table['Asian']['Sum'] += 1
    elif(data['InsuredInfo_8'][i] == 'Hispanic'):
        table['Hispanic'][data['Response'][i]] += 1
        table['Hispanic']['Sum'] += 1
    elif(data['InsuredInfo_8'][i] == 'Latino'):
        table['Latino'][data['Response'][i]] += 1
        table['Latino']['Sum'] += 1
    else:
        table['Other'][data['Response'][i]] += 1
        table['Other']['Sum'] += 1
    table['Sum'][data['Response'][i]] += 1
    table['Sum']['Sum'] += 1
table

,Caucasian,Black,Arabic,African-American,Asian,Hispanic,Latino,Other,Sum
1,122,440,2,63,51,20,51,5409,6158
2,104,350,4,47,48,23,77,5842,6495
3,11,59,0,16,8,3,10,902,1009
4,20,79,0,25,6,12,16,1262,1420
5,85,429,1,62,36,18,65,4694,5390
6,275,749,2,134,66,32,162,9716,11136
7,130,531,0,97,61,36,130,6961,7946
8,299,1164,6,314,121,126,315,16982,19327
Sum,1046,3801,15,758,397,270,826,51768,58881


In [23]:
for i in range(X_test.shape[0]):
    if(X_test[i][24] == 11323):
        table['Caucasian'][table_predict[i]] += 1
        table['Caucasian']['Sum'] += 1
    elif(X_test[i][24] == 11317):
        table['Black'][table_predict[i]] += 1
        table['Black']['Sum'] += 1
    elif(X_test[i][24] == 11313):
        table['Arabic'][table_predict[i]] += 1
        table['Arabic']['Sum'] += 1
    elif(X_test[i][24] == 11334):
        table['African-American'][table_predict[i]] += 1
        table['African-American']['Sum'] += 1
    elif(X_test[i][24] == 11314):
        table['Asian'][table_predict[i]] += 1
        table['Asian']['Sum'] += 1
    elif(X_test[i][24] == 11332):
        table['Hispanic'][table_predict[i]] += 1
        table['Hispanic']['Sum'] += 1
    elif(X_test[i][24] == 11312):
        table['Latino'][table_predict[i]] += 1
        table['Latino']['Sum'] += 1
    else:
        table['Other'][table_predict[i]] += 1
        table['Other']['Sum'] += 1
    table['Sum'][table_predict[i]] += 1
    table['Sum']['Sum'] += 1
table

,Caucasian,Black,Arabic,African-American,Asian,Hispanic,Latino,Other,Sum
1,23,74,0,9,8,5,17,1979,2115
2,17,56,0,13,7,4,8,1858,1963
3,0,0,0,0,0,0,0,12,12
4,1,3,0,1,1,0,0,112,118
5,2,20,0,7,2,0,1,615,647
6,91,284,0,66,31,3,56,6384,6915
7,16,48,0,7,15,2,11,1844,1943
8,98,513,4,130,46,53,124,14925,15893
Sum,248,998,4,233,110,67,217,27729,29606


In [24]:
T = 0

for g in table.keys():
    for i in range(1,9):
        Eij = (table['Sum'][i] * table[g]['Sum'])/table['Sum']['Sum']
        T += (table[g][i] - Eij)**2/Eij
        
T

112.00080335450457